In [1]:
import torch
import cv2

In [2]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model

Using cache found in /Users/vatsal007/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-10 Python-3.8.16 torch-2.1.0.dev20230302 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [73]:
cap = cv2.VideoCapture('yolovid.mp4')
carcount = 0
while(cap.isOpened()):
      
# Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        
        #detecting any object
        img = []
        img.append(frame)
        results = model(img).pandas().xyxy
        vehicles = []

        #detecting vehicles
        for result in results: #result is a dataframe
            if(result.empty == 0):
                for i in result.index:
                    if(result['name'][i] == 'car' or result['name'][i] == 'truck' or result['name'][i] == 'motorcycle' or result['name'][i] == 'bicycle'):
                        vehicle = []
                        vehicle.append(result['xmin'][i])
                        vehicle.append(result['ymin'][i])
                        vehicle.append(result['xmax'][i])
                        vehicle.append(result['ymax'][i])
                        vehicle.append(result['confidence'][i])
                        vehicle.append(result['name'][i])
                        vehicles.append(vehicle)

        frame = cv2.putText(frame, 'vehicles in frame: '+str(len(vehicles)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (255, 0, 0), 2, cv2.LINE_AA) 
        

        #bounding boxes for the vehicles
        for vehicle in vehicles:
            start_point = (int(vehicle[0]), int(vehicle[1]))
            end_point = (int(vehicle[2]), int(vehicle[3]))
            color = (255, 0, 0)
            thickness = 2
            vehicleconfidence = vehicle[4]
            vehicletype = vehicle[5]
            
            frame = cv2.putText(frame, str(vehicletype)+' : '+str(round(vehicleconfidence*100,2)), (int(vehicle[0]), int(vehicle[1]-6)), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0, 0, 255), 2, cv2.LINE_AA) 
            frame = cv2.rectangle(frame, start_point, end_point, color, thickness)      

        #implementing line of detection
        lineOfDetectionY = int(frame.shape[0]/2)

        frame = cv2.putText(frame, 'line of detection', (5,lineOfDetectionY-10), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0, 255, 255), 2, cv2.LINE_AA) 
        frame = cv2.line(frame, (0,lineOfDetectionY), (frame.shape[1],lineOfDetectionY), (0, 255, 255), thickness)     
        cv2.imshow('Frame', frame)
            

    # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
  
# Break the loop
    else:
        break

In [20]:
results = model(img)

In [28]:
len(results.pandas().xyxy)

228